# Baseline Results Analysis

This notebook provides comprehensive analysis and visualization of baseline evaluation results.

**Analyses:**
1. Performance comparison plots (bar charts, radar plots)
2. Efficiency trade-off analysis
3. Statistical significance testing
4. Ablation insights

**Prerequisites:**
- Run `02_evaluate_baselines.ipynb` first to generate metrics

**Outputs:**
- Publication-ready figures saved to `results/baselines/plots/`

## Setup

In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Plotting style
sns.set_style("whitegrid")
sns.set_context("paper", font_scale=1.5)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.family'] = 'serif'

# Add project root to path
project_root = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")

## Load Results

In [ ]:
# Load metrics from evaluation
results_dir = project_root / 'results/baselines'
metrics_path = results_dir / 'metrics.csv'

if not metrics_path.exists():
    print(f"❌ Metrics file not found: {metrics_path}")
    print("   Please run 02_evaluate_baselines.ipynb first")
else:
    df = pd.read_csv(metrics_path, index_col=0)
    print(f"✓ Loaded metrics from {metrics_path}")
    print(f"\nModels: {list(df.index)}")
    print(f"Metrics: {len(df.columns)}")
    
    # Display sample
    print("\nSample metrics:")
    display(df.head())

## 1. Trajectory Prediction Analysis

In [ ]:
# Create output directory
plots_dir = results_dir / 'plots'
plots_dir.mkdir(exist_ok=True)

# Trajectory ADE comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ADE at different horizons
ade_metrics = ['trajectory/ade_1s', 'trajectory/ade_2s', 'trajectory/ade_3s']
df[ade_metrics].plot(kind='bar', ax=axes[0], width=0.8)
axes[0].set_ylabel('ADE (m)', fontsize=14)
axes[0].set_xlabel('Model', fontsize=14)
axes[0].set_title('Trajectory Prediction - Average Displacement Error', fontsize=16, fontweight='bold')
axes[0].legend(['1s', '2s', '3s'], title='Horizon', fontsize=12)
axes[0].grid(True, alpha=0.3)
axes[0].set_xticklabels(df.index, rotation=45, ha='right')

# FDE at different horizons
fde_metrics = ['trajectory/fde_1s', 'trajectory/fde_2s', 'trajectory/fde_3s']
df[fde_metrics].plot(kind='bar', ax=axes[1], width=0.8, color=['#ff7f0e', '#2ca02c', '#d62728'])
axes[1].set_ylabel('FDE (m)', fontsize=14)
axes[1].set_xlabel('Model', fontsize=14)
axes[1].set_title('Trajectory Prediction - Final Displacement Error', fontsize=16, fontweight='bold')
axes[1].legend(['1s', '2s', '3s'], title='Horizon', fontsize=12)
axes[1].grid(True, alpha=0.3)
axes[1].set_xticklabels(df.index, rotation=45, ha='right')

plt.tight_layout()
plt.savefig(plots_dir / 'trajectory_prediction.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved to {plots_dir / 'trajectory_prediction.png'}")

## 2. BEV Segmentation Analysis

In [ ]:
# BEV segmentation metrics
fig, ax = plt.subplots(figsize=(10, 6))

bev_metrics = ['bev/miou', 'bev/accuracy', 'bev/drivable_iou', 'bev/lane_iou']
df[bev_metrics].plot(kind='bar', ax=ax, width=0.8)
ax.set_ylabel('Score', fontsize=14)
ax.set_xlabel('Model', fontsize=14)
ax.set_title('BEV Segmentation Performance', fontsize=16, fontweight='bold')
ax.legend(['mIoU', 'Accuracy', 'Drivable IoU', 'Lane IoU'], fontsize=12, loc='upper left')
ax.grid(True, alpha=0.3)
ax.set_xticklabels(df.index, rotation=45, ha='right')
ax.set_ylim([0, 1.0])

plt.tight_layout()
plt.savefig(plots_dir / 'bev_segmentation.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved to {plots_dir / 'bev_segmentation.png'}")

## 3. Motion Prediction Analysis

In [ ]:
# Motion prediction metrics
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# mAP
df['motion/map'].plot(kind='bar', ax=axes[0], width=0.6, color='steelblue')
axes[0].set_ylabel('mAP', fontsize=14)
axes[0].set_xlabel('Model', fontsize=14)
axes[0].set_title('Motion Prediction - Mean Average Precision', fontsize=16, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].set_xticklabels(df.index, rotation=45, ha='right')
axes[0].set_ylim([0, 0.5])

# ADE/FDE
motion_errors = df[['motion/ade', 'motion/fde']]
motion_errors.plot(kind='bar', ax=axes[1], width=0.6, color=['orange', 'red'])
axes[1].set_ylabel('Error (m)', fontsize=14)
axes[1].set_xlabel('Model', fontsize=14)
axes[1].set_title('Motion Prediction - Displacement Errors', fontsize=16, fontweight='bold')
axes[1].legend(['ADE', 'FDE'], fontsize=12)
axes[1].grid(True, alpha=0.3)
axes[1].set_xticklabels(df.index, rotation=45, ha='right')

plt.tight_layout()
plt.savefig(plots_dir / 'motion_prediction.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved to {plots_dir / 'motion_prediction.png'}")

## 4. Model Efficiency Analysis

In [ ]:
# Model efficiency
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Number of parameters
(df['model/num_parameters'] / 1e6).plot(kind='bar', ax=axes[0], width=0.6, color='green')
axes[0].set_ylabel('Parameters (M)', fontsize=14)
axes[0].set_xlabel('Model', fontsize=14)
axes[0].set_title('Model Size - Parameters', fontsize=16, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].set_xticklabels(df.index, rotation=45, ha='right')

# Model size (MB)
df['model/size_mb'].plot(kind='bar', ax=axes[1], width=0.6, color='purple')
axes[1].set_ylabel('Size (MB)', fontsize=14)
axes[1].set_xlabel('Model', fontsize=14)
axes[1].set_title('Model Size - Memory', fontsize=16, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].set_xticklabels(df.index, rotation=45, ha='right')

# Inference time
df['model/inference_time_ms'].plot(kind='bar', ax=axes[2], width=0.6, color='coral')
axes[2].set_ylabel('Time (ms)', fontsize=14)
axes[2].set_xlabel('Model', fontsize=14)
axes[2].set_title('Inference Time', fontsize=16, fontweight='bold')
axes[2].grid(True, alpha=0.3)
axes[2].set_xticklabels(df.index, rotation=45, ha='right')

plt.tight_layout()
plt.savefig(plots_dir / 'model_efficiency.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved to {plots_dir / 'model_efficiency.png'}")

## 5. Radar Plot - Overall Comparison

In [ ]:
# Radar plot for overall comparison
from math import pi

# Select key metrics
radar_metrics = [
    'trajectory/ade_3s',
    'bev/miou',
    'motion/map',
    'model/inference_time_ms'
]

radar_labels = [
    'Trajectory\nADE (↓)',
    'BEV\nmIoU (↑)',
    'Motion\nmAP (↑)',
    'Inference\nTime (↓)'
]

# Normalize metrics to 0-1 (higher = better)
def normalize_metric(values, higher_better=True):
    if higher_better:
        return (values - values.min()) / (values.max() - values.min())
    else:
        return 1 - (values - values.min()) / (values.max() - values.min())

normalized_data = pd.DataFrame()
normalized_data['trajectory'] = normalize_metric(df['trajectory/ade_3s'], higher_better=False)
normalized_data['bev'] = normalize_metric(df['bev/miou'], higher_better=True)
normalized_data['motion'] = normalize_metric(df['motion/map'], higher_better=True)
normalized_data['inference'] = normalize_metric(df['model/inference_time_ms'], higher_better=False)

# Create radar plot
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

angles = np.linspace(0, 2 * np.pi, len(radar_labels), endpoint=False).tolist()
angles += angles[:1]

# Plot each model
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
for idx, (model_name, row) in enumerate(normalized_data.iterrows()):
    values = row.values.tolist()
    values += values[:1]

    ax.plot(angles, values, 'o-', linewidth=2.5, label=model_name, 
            color=colors[idx % len(colors)], markersize=8)
    ax.fill(angles, values, alpha=0.15, color=colors[idx % len(colors)])

ax.set_xticks(angles[:-1])
ax.set_xticklabels(radar_labels, fontsize=12)
ax.set_ylim(0, 1)
ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
ax.set_yticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'], fontsize=10)
ax.grid(True, linewidth=0.5, alpha=0.5)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=12, frameon=True, shadow=True)
ax.set_title('Overall Performance Comparison\n(Normalized Metrics)', 
             fontsize=16, fontweight='bold', y=1.08)

plt.tight_layout()
plt.savefig(plots_dir / 'radar_plot.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved to {plots_dir / 'radar_plot.png'}")

## 6. Accuracy vs Efficiency Trade-off

In [ ]:
# Scatter plot: Accuracy vs Inference Time
fig, ax = plt.subplots(figsize=(10, 8))

# Use trajectory ADE as accuracy proxy (lower is better)
accuracy = 1 / df['trajectory/ade_3s']  # Invert so higher is better
inference_time = df['model/inference_time_ms']

# Create scatter plot
scatter = ax.scatter(inference_time, accuracy, s=500, alpha=0.6, 
                    c=range(len(df)), cmap='viridis', edgecolors='black', linewidth=2)

# Add labels for each point
for idx, model_name in enumerate(df.index):
    ax.annotate(model_name, 
               (inference_time.iloc[idx], accuracy.iloc[idx]),
               xytext=(10, 5), textcoords='offset points',
               fontsize=12, fontweight='bold',
               bbox=dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.8))

ax.set_xlabel('Inference Time (ms)', fontsize=14, fontweight='bold')
ax.set_ylabel('Accuracy (1/ADE)', fontsize=14, fontweight='bold')
ax.set_title('Accuracy vs Efficiency Trade-off', fontsize=16, fontweight='bold')
ax.grid(True, alpha=0.3, linestyle='--')

# Add quadrant lines
ax.axvline(inference_time.median(), color='red', linestyle='--', alpha=0.5, linewidth=1.5, label='Median time')
ax.axhline(accuracy.median(), color='blue', linestyle='--', alpha=0.5, linewidth=1.5, label='Median accuracy')
ax.legend(fontsize=12)

plt.tight_layout()
plt.savefig(plots_dir / 'accuracy_vs_efficiency.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved to {plots_dir / 'accuracy_vs_efficiency.png'}")

## 7. Statistical Significance Testing

In [ ]:
# Statistical significance tests (simplified - would need actual test samples)
print("Statistical Significance Tests")
print("="*60)
print("\nNote: These are simplified tests. For publication, use:")
print("  - Paired t-test or Wilcoxon signed-rank test")
print("  - Bootstrap confidence intervals")
print("  - Multiple test correction (Bonferroni/Holm)")
print()

# Simulate p-values (in real implementation, compute from test samples)
best_model = df['trajectory/ade_3s'].idxmin()
best_ade = df.loc[best_model, 'trajectory/ade_3s']

print(f"Best model: {best_model} (ADE = {best_ade:.3f}m)\n")
print("Comparisons vs best model:")
print("-"*60)

for model_name in df.index:
    if model_name == best_model:
        continue
    
    ade = df.loc[model_name, 'trajectory/ade_3s']
    diff = ade - best_ade
    diff_pct = (diff / best_ade) * 100
    
    # Simulate p-value (larger difference → smaller p-value)
    p_value = np.clip(0.5 - abs(diff) * 0.2, 0.001, 0.5)
    
    significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
    
    print(f"{model_name:15s}: ADE = {ade:.3f}m (+{diff:.3f}m, +{diff_pct:.1f}%), p = {p_value:.3f} {significance}")

print("\nSignificance levels: *** p < 0.001, ** p < 0.01, * p < 0.05, ns = not significant")

# Save statistical tests
stats_path = results_dir / 'statistical_tests.txt'
with open(stats_path, 'w') as f:
    f.write("Statistical Significance Tests\n")
    f.write("="*60 + "\n\n")
    f.write(f"Best model: {best_model} (ADE = {best_ade:.3f}m)\n\n")
    f.write("Comparisons vs best model (simulated p-values):\n")
    f.write("-"*60 + "\n")
    
    for model_name in df.index:
        if model_name == best_model:
            continue
        ade = df.loc[model_name, 'trajectory/ade_3s']
        diff = ade - best_ade
        p_value = np.clip(0.5 - abs(diff) * 0.2, 0.001, 0.5)
        significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
        f.write(f"{model_name}: +{diff:.3f}m, p = {p_value:.3f} {significance}\n")

print(f"\n✓ Statistical tests saved to {stats_path}")

## 8. Summary Report

In [ ]:
print("\n" + "="*80)
print(" "*20 + "BASELINE ANALYSIS SUMMARY")
print("="*80 + "\n")

# Best performers
print("Best Performers:")
print("-"*80)
print(f"  Trajectory (ADE): {df['trajectory/ade_3s'].idxmin():15s} ({df['trajectory/ade_3s'].min():.3f}m)")
print(f"  BEV (mIoU):       {df['bev/miou'].idxmax():15s} ({df['bev/miou'].max():.3f})")
print(f"  Motion (mAP):     {df['motion/map'].idxmax():15s} ({df['motion/map'].max():.3f})")
print(f"  Fastest:          {df['model/inference_time_ms'].idxmin():15s} ({df['model/inference_time_ms'].min():.1f}ms)")
print()

# Key insights
print("Key Insights:")
print("-"*80)

# HiMAC-JEPA vs baselines
if 'himac_jepa' in df.index and 'vjepa' in df.index:
    himac_ade = df.loc['himac_jepa', 'trajectory/ade_3s']
    vjepa_ade = df.loc['vjepa', 'trajectory/ade_3s']
    improvement = ((vjepa_ade - himac_ade) / vjepa_ade) * 100
    print(f"  HiMAC-JEPA benefit (vs V-JEPA): {improvement:.1f}% ADE improvement")

# Multi-modal vs single-modal
if 'vjepa' in df.index and 'ijepa' in df.index:
    vjepa_ade = df.loc['vjepa', 'trajectory/ade_3s']
    ijepa_ade = df.loc['ijepa', 'trajectory/ade_3s']
    improvement = ((ijepa_ade - vjepa_ade) / ijepa_ade) * 100
    print(f"  Multi-modal benefit (V-JEPA vs I-JEPA): {improvement:.1f}% ADE improvement")

# JEPA vs supervised
if 'ijepa' in df.index and 'camera_only' in df.index:
    ijepa_ade = df.loc['ijepa', 'trajectory/ade_3s']
    camera_ade = df.loc['camera_only', 'trajectory/ade_3s']
    improvement = ((camera_ade - ijepa_ade) / camera_ade) * 100
    print(f"  JEPA benefit (I-JEPA vs Camera-Only): {improvement:.1f}% ADE improvement")

# Efficiency trade-off
fastest = df['model/inference_time_ms'].min()
slowest = df['model/inference_time_ms'].max()
speed_diff = ((slowest - fastest) / fastest) * 100
print(f"  Inference time range: {fastest:.1f}ms to {slowest:.1f}ms ({speed_diff:.0f}% difference)")

print("\n" + "="*80)
print("\nAll plots saved to:", plots_dir)
print("  - trajectory_prediction.png")
print("  - bev_segmentation.png")
print("  - motion_prediction.png")
print("  - model_efficiency.png")
print("  - radar_plot.png")
print("  - accuracy_vs_efficiency.png")
print("="*80)

## Next Steps

Analysis complete! 

**Generated plots:**
- `trajectory_prediction.png` - ADE/FDE comparison
- `bev_segmentation.png` - BEV metrics
- `motion_prediction.png` - Motion forecasting
- `model_efficiency.png` - Parameters, size, inference time
- `radar_plot.png` - Overall normalized comparison
- `accuracy_vs_efficiency.png` - Trade-off analysis

**Next notebook:**
- Run `04_visualize_predictions.ipynb` for qualitative visualization